# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [4]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一片宁静的自然景观。画面中有一条木栈道穿过茂密的绿草地，栈道延伸向远方，给人一种探索的感觉。草地生机勃勃，周围有树木和灌木，表现出自然的丰富和多样性。天空中有柔和的云彩，阳光洒在大地上，营造出一种宁静而舒适的氛围。这幅图非常适合用来表达与自然亲密接触的感觉，以及探索和放松的主题。', role='assistant', function_call=None, tool_calls=None))


In [5]:
response.choices[0].message.content

'这幅图描绘了一片宁静的自然景观。画面中有一条木栈道穿过茂密的绿草地，栈道延伸向远方，给人一种探索的感觉。草地生机勃勃，周围有树木和灌木，表现出自然的丰富和多样性。天空中有柔和的云彩，阳光洒在大地上，营造出一种宁静而舒适的氛围。这幅图非常适合用来表达与自然亲密接触的感觉，以及探索和放松的主题。'

### 封装成一个函数 query_image_description

In [6]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [13]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默图片，展示了两种极端不同的狗的卡通形象。左边的狗被设计得非常肌肉发达，类似于人类健美选手，带有浓厚的夸张元素。对于这只狗的描述是“16岁的我，工作后的我”，意味着这个形象代表了年轻时候的充满希望和活力。

右边的狗看起来较为普通，有点憔悴和无奈的样子，体现了工作后的疲惫和现实压力。这种对比强调了时间和生活环境对个体的影响，以及青春期与成年后自我形象之间的巨大差异。

整体上，这幅图通过幽默且夸张的方式，来表达人们对于年轻和成熟阶段的感受和对比。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    api_key='sk-iIWdN3LYHyHnD67P83E8E2Bf3e5f44F0Ac03E13175Af0a32',         
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt, max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [2]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg", "请描述这张图")
print(content)

这张图表展示了1980年至2020年期间四个国家的国内生产总值（GDP）比较。图中使用了不同颜色的线条代表不同国家：

- **美国**（蓝色线条）：从1980年开始，GDP持续上升，并在2020年达到最高点，显示出强劲的经济增长。
- **中国**（红色线条）：从1990年代开始快速增长，特别是在2000年后，增速显著，逐渐接近美国的水平。
- **日本**（紫色线条）：其GDP在1980年代和1990年代有较高水平，但在2000年后增速放缓，整体趋于平稳。
- **德国**（绿色线条）：GDP相对较稳定，增速缓慢，波动较小。

整个图表清晰地展示了这四个经济体的增长趋势和相对位置变化。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [3]:
content = query_base64_image_description("./images/handwriting_0.jpg", "请描述这张图")
print(content)

这是一页手写的笔记，标题为“PEFT（Parameter-Efficient Fine-Tuning）”。内容主要涉及不同的微调方法，如：

1. **Prompt Tuning**：使用小模型与预训练模型结合，描述了输入和输出的关系。
2. **Prefix Tuning**：介绍了如何通过变换器的编码器和解码器处理输入。
3. **LoRA（Low-Rank Adaptation）**：一个较大模型中添加微调的公式，涉及到矩阵运算。
4. **QLoRA**：可能是对LoRA的某种变体或改进，并提到相关模型的大小。

整个页面包含了数学公式和简要的说明，表达了对这几种微调方法的理解。背景是一本笔记本，页面为淡黄色，手写内容清晰可见。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [8]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这是一页手写的笔记，标题为“PEFT（Parameter-Efficient Fine-Tuning）”。内容主要涉及不同的微调方法，如：

1. **Prompt Tuning**：使用小模型与预训练模型结合，描述了输入和输出的关系。
2. **Prefix Tuning**：介绍了如何通过变换器的编码器和解码器处理输入。
3. **LoRA（Low-Rank Adaptation）**：一个较大模型中添加微调的公式，涉及到矩阵运算。
4. **QLoRA**：可能是对LoRA的某种变体或改进，并提到相关模型的大小。

整个页面包含了数学公式和简要的说明，表达了对这几种微调方法的理解。背景是一本笔记本，页面为淡黄色，手写内容清晰可见。

![](./images/handwriting_1.jpg)

In [11]:
content = query_base64_image_description("./images/handwriting_1.jpg", "请描述这张图")
display(Markdown(content))

这张图展示了一本笔记本的打开页面，内容为有关模型微调、提示调优和多模态指令等主题的笔记。页面上有不同的标题和子标题，包含了PEFT（参数有效微调）、Prompt Tuning（提示调优）等多种方法的概述，以及相关的研究年份和引用。笔记中还提到了各种技术和方法的缩写，如LoRA、QLoRA等，并引入了前缀调优与适配器的概念，内容较为技术性，适合对机器学习和自然语言处理有研究的人士。页面的书写风格随意，似乎是个人的学习记录。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [16]:
content = query_base64_image_description("./images/gpt-4v.jpg", "请描述这张图, 并且对图中的提到的问题给出答案")
display(Markdown(content))

这是一张手写的思维导图，主题是关于“Langchain”的讨论。图中左侧是“Langchain”，并有多个分支和主题相关的内容。

从左到右的主要内容包括：

1. **Langchain**：中心主题。
2. **与 Langchain 相关的需求**。
3. **关于 Langchain 的核心功能**：
   - **Mode I/O**：与输入输出相关的功能。
   - **Chains**：涉及链式结构的组件。
   - **Data Connection**：用于数据连接的方法。
   - **Vector Store**：存储向量的数据结构。
   - **Retriever**：检索相关数据的功能。

在图的底部，有一个问题：“Langchain 是如何适应非专业开始者的？也请给一些具体的示例说明。”

### 问题的答案:
Langchain 通过提供简洁的接口和文档，使非专业用户能够轻松开始使用。对于具体示例：

1. **简化的API**：Langchain的设计为用户提供了预设的功能，允许用户通过少量代码实现复杂的任务。
2. **示例项目**：提供示例代码或项目，用户可以直接运行并根据自己的需求进行修改。
3. **模块化结构**：用户可以根据需要选择使用不同的功能模块，如数据连接、检索和存储等，便于按需学习。

总体而言，Langchain 通过易用性和具体的实例帮助非专业用户快速上手。